In [1]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd

# dataset
data = pd.read_csv('../data/weather/weather.csv')

# the list of unique capitals. Some capitals where duplicated
capitals = data['capital'].unique()

# create a dash app in python
app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("Weather Data"),
    html.Label("Select a capital:"),
    dcc.Dropdown(
        id='capital-dropdown',
        options=[{'label': capital, 'value': capital} for capital in capitals],
        value='Mariehamn'
    ),
    html.Label("Select a date range:"),
    dcc.DatePickerRange(
        id='date-range-picker',
        start_date='2016-01-01',
        end_date='2020-12-31'
    ),
    dcc.Graph(id='temperature-plot'),
    dcc.Graph(id='rain-plot'),
    dcc.Graph(id='snow-plot'),
    html.Div(id='statistics')  # container for statistics
])

@app.callback(
    [Output('temperature-plot', 'figure'),
     Output('rain-plot', 'figure'),
     Output('snow-plot', 'figure'),
     Output('statistics', 'children')],  # output for statistics
    [Input('capital-dropdown', 'value'),
     Input('date-range-picker', 'start_date'),
     Input('date-range-picker', 'end_date')]
)
def update_plots(selected_capital, start_date, end_date):
    # filter the data for the selected capital and date range
    filtered_data = data[(data['capital'] == selected_capital) & (data['date'] >= start_date) & (data['date'] <= end_date)]

    # create a temperature line plot
    temperature_fig = px.line(filtered_data, x='date', y='temp_mean', title=f'Temperature in {selected_capital}')

    # create a rainfall line plot
    rain_fig = px.line(filtered_data, x='date', y='rain', title=f'Rainfall in {selected_capital}')

    # create a snowfall line plot
    snow_fig = px.line(filtered_data, x='date', y='snow', title=f'Snowfall in {selected_capital}')

    # calculate statistics not needed but I was just interesting in these statistics
    mean_temp = filtered_data['temp_mean'].mean()
    max_temp = filtered_data['temp_mean'].max()
    min_temp = filtered_data['temp_mean'].min()

    mean_rain = filtered_data['rain'].mean()
    max_rain = filtered_data['rain'].max()
    min_rain = filtered_data['rain'].min()

    mean_snow = filtered_data['snow'].mean()
    max_snow = filtered_data['snow'].max()
    min_snow = filtered_data['snow'].min()

    # create statistics text
    statistics_text = html.Div([
        html.H2("Statistics"),
        html.P(f"Mean Temperature: {mean_temp:.2f}°C"),
        html.P(f"Max Temperature: {max_temp:.2f}°C"),
        html.P(f"Min Temperature: {min_temp:.2f}°C"),
        html.P(f"Mean Rainfall: {mean_rain:.2f} mm"),
        html.P(f"Max Rainfall: {max_rain:.2f} mm"),
        html.P(f"Min Rainfall: {min_rain:.2f} mm"),
        html.P(f"Mean Snowfall: {mean_snow:.2f} mm"),
        html.P(f"Max Snowfall: {max_snow:.2f} mm"),
        html.P(f"Min Snowfall: {min_snow:.2f} mm")
    ])

    return temperature_fig, rain_fig, snow_fig, statistics_text

if __name__ == '__main__':
    app.run_server(debug=True, port=8053) #port=8053 was needed to run the code on my laptop
